In [1]:
import os
os.chdir('..')

In [2]:
import torch
import numpy as np
torch.manual_seed(1992)
# torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(1992)

In [3]:
import utils
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from scipy import linalg

In [4]:
trainX, trainy, testX, testy = utils.load_dataset()

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 1) (2947, 128, 9) (2947, 1)


In [5]:
trainX = torch.tensor(trainX.transpose(0,2,1), dtype=torch.float).cuda()
trainy = torch.tensor(trainy, dtype=torch.float).cuda()
testX = torch.tensor(testX.transpose(0,2,1), dtype=torch.float).cuda()
testy = torch.tensor(testy, dtype=torch.float).cuda()
testy_c = testy.max(dim=1)[1]

In [64]:
verbose, epochs, batch_size = True, 500, 32
n_timesteps, n_features, n_outputs = trainX.shape[2], trainX.shape[1], trainy.shape[1]
n_timesteps, n_features, n_outputs

(128, 9, 6)

In [7]:
def build_tensor(size, kernel_size):
    coords = []
    vals = []
    for k in range(size):
        for j in range(kernel_size):
            i = k - j + 1
            if k in range(size) and i in range(size) and j in range(kernel_size):
                coords.append([k, i, j])
                vals.append(1)

    i = torch.LongTensor(coords)
    v = torch.FloatTensor(vals)
    return torch.sparse.FloatTensor(i.t(), v, torch.Size([size, size, kernel_size]))

In [8]:
# def apply(func, M, s):
#     tList = [func(m, s) for m in torch.unbind(M) ]
#     res = torch.stack(tList, dim=0)

#     return res 

In [9]:
# def toeplitz(kernel, input_size):
#     half = int(np.ceil(len(kernel)/2))
#     fst = torch.flip(kernel[0:half], [0])
#     sec = kernel[half-1:]
#     t = linalg.toeplitz(np.pad(sec, (0, input_size - len(sec))), np.pad(fst, (0, input_size - len(fst))))
#     return torch.tensor(t)

## Default topology

In [10]:
# class Dummy(nn.Module):
#     def __init__(self, in_channels, out_channels, kernel_size):
#         super(Dummy, self).__init__()
#         self.in_channels = in_channels
#         self.out_channels = out_channels
#         self.kernel_size = kernel_size
#         self.weights = torch.eye(self.num_features).expand((self.size, self.num_features, self.num_features)).float()#.cuda()
        
#     def forward(self, x):
#         print(x.shape, self.weights.shape)
#         # X : (output_size, batch_size, num_features)
#         return torch.bmm(x, self.weights)

In [886]:
batch = 10
F = 3
t = 5
OUT = 6
k = 3

In [958]:
X = torch.rand(batch, F, t)

In [959]:
X.shape

torch.Size([10, 3, 5])

In [926]:
w = torch.rand(OUT, k).T

In [927]:
T = build_tensor(t, k).to_dense().expand(F, t, t, k)

In [928]:
T.shape

torch.Size([3, 5, 5, 3])

In [932]:
w.shape

torch.Size([3, 6])

In [954]:
W = torch.matmul(T, w)#.permute(2, 0, 1)

In [955]:
W.shape

torch.Size([3, 5, 5, 6])

In [956]:
W[0, :, :, 0]

tensor([[0.7851, 0.2857, 0.0000, 0.0000, 0.0000],
        [0.0254, 0.7851, 0.2857, 0.0000, 0.0000],
        [0.0000, 0.0254, 0.7851, 0.2857, 0.0000],
        [0.0000, 0.0000, 0.0254, 0.7851, 0.2857],
        [0.0000, 0.0000, 0.0000, 0.0254, 0.7851]])

In [960]:
X[0, 0, :]

tensor([0.9566, 0.1663, 0.1594, 0.1825, 0.0993])

In [961]:
torch.matmul(W[0, :, :, 0],  X[0, 0, :])

tensor([0.7985, 0.2004, 0.1815, 0.1757, 0.0826])

In [962]:
torch.tensordot(X, W).shape

torch.Size([10, 5, 6])

In [965]:
torch.tensordot(X, W)[0, :, 0]

tensor([1.4117, 1.2699, 1.5082, 1.5161, 0.7790])

In [627]:
w.shape

torch.Size([64, 9, 3])

In [628]:
T.shape

torch.Size([9, 3, 128, 128])

In [629]:
W = torch.tensordot(w, T).expand(9, 64, 128, 128).permute(0,2,1,3)

In [630]:
W.shape

torch.Size([9, 128, 64, 128])

In [632]:
X.shape

torch.Size([1000, 9, 128])

In [633]:
torch.tensordot(X, W).shape

torch.Size([1000, 64, 128])

In [625]:
torch.tensordot(X, W).shape

torch.Size([1000, 64, 128])

In [321]:
cnn = nn.Conv1d(9, 64, 3)

In [322]:
type(cnn)

torch.nn.modules.conv.Conv1d

In [323]:
cnn.weight.shape

torch.Size([64, 9, 3])

In [654]:
class CustomConv1d(nn.Module):
    def __init__(self, size, in_channels, out_channels, kernel_size):
        super(CustomConv1d, self).__init__()
        self.size = size
        self.num_features = in_channels
        self.kernel_size = kernel_size
        self.out_channels = out_channels
        y = 1.0 / np.sqrt(in_channels)
        sampler = torch.distributions.Uniform(low=-y, high=y) 
        A = sampler.sample((self.out_channels, self.num_features, self.kernel_size)).cuda()
        self.T = build_tensor(self.size, self.kernel_size).to_dense().expand(self.num_features, size, size, kernel_size).permute(0, 3, 1, 2).cuda()
#         self.weights = nn.Parameter(apply(toeplitz, A, self.size).expand(self.num_features, self.out_channels, self.size, self.size).permute(0,2,1,3).clone().cuda())
        self.weights = nn.Parameter(A)
        self.weights.requires_grad = True

    def forward(self, x):
        
#         print("T: ", self.T.shape)
        W = torch.tensordot(self.weights, self.T).expand(self.num_features, self.out_channels, self.size, self.size).permute(0,2,1,3)
#         W = torch.matmul(self.T, self.weights).expand(self.num_features, self.size, self.size, self.out_channels).view(self.num_features, self.size, self.out_channels, self.size)#permute(0,1,3,2)
#         print("X: ", x.shape)
#         print("W: ", W.shape)
        x = torch.tensordot(x, W)
        
#         end.record()
#         torch.cuda.synchronize()

#         print("TENSORDOT: ", start.elapsed_time(end))
        
        return x

In [655]:
# class CustomConv1d(nn.Module):
#     def __init__(self, size, in_channels, out_channels, kernel_size):
#         super(CustomConv1d, self).__init__()
#         self.size = size
#         self.num_features = in_channels
#         self.kernel_size = kernel_size
#         self.out_channels = out_channels
#         y = 1.0 / np.sqrt(in_channels)
#         sampler = torch.distributions.Uniform(low=-y, high=y) 
#         A = sampler.sample((self.out_channels, self.kernel_size))

#         self.weights = nn.Parameter(apply(toeplitz, A, self.size).expand(self.num_features, self.out_channels, self.size, self.size).clone().permute(0,2,1,3).cuda())
#         self.weights.requires_grad = True
#         print("CALL")

#     def forward(self, x):
#         return torch.tensordot(x, self.weights)

In [656]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = CustomConv1d(n_timesteps, n_features, 64, 3)
        self.relu1 = nn.Tanh()
        self.conv2 = CustomConv1d(n_timesteps, 64, 64, 3)
        self.relu2 = nn.Tanh()
        self.drop = nn.Dropout(0.5)
        self.maxp = nn.AvgPool1d(2)
        self.flat = nn.Flatten(1,-1)
        self.dense1 = nn.Linear(4096, 100)
        self.relu3 = nn.Tanh()
        self.dense2 = nn.Linear(100, 6)
        
    def forward(self, x):
#         start = torch.cuda.Event(enable_timing=True)
#         end = torch.cuda.Event(enable_timing=True)

#         start.record()
        x = self.conv1(x)

#         end.record()
#         torch.cuda.synchronize()

#         print("CUSTOM_LAYER: ", start.elapsed_time(end))

        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.drop(x)
        x = self.maxp(x)
        x = self.flat(x)
        
#         start = torch.cuda.Event(enable_timing=True)
#         end = torch.cuda.Event(enable_timing=True)

#         start.record()

        x = self.dense1(x)
        
#         end.record()
#         torch.cuda.synchronize()

#         print("DENSE: ", start.elapsed_time(end))

        x = self.relu3(x)
        x = self.dense2(x)
        output = F.softmax(x, dim=1)
        return output

In [657]:
use_cuda = torch.cuda.is_available()

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'batch_size': batch_size}

In [658]:
model = Net().to(device)
optimizer = optim.Adam(model.parameters())

trainy_c = trainy.max(dim=1)[1]
print(len(list(model.parameters())))
for t in range(epochs):
    y_pred = model(trainX)
    criterion = nn.NLLLoss()
    loss = criterion(torch.log(y_pred), trainy_c)
    acc_train = float((trainy_c.long().eq(torch.max(y_pred, 1)[1].long())).sum())/(float(trainy_c.shape[0]))
    print("Epoch: {}\tLoss: {}\tAcc: {}".format(t+1, loss, acc_train))
    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

6
Epoch: 1	Loss: 1.8585984706878662	Acc: 0.11887921653971709
Epoch: 2	Loss: 2.5444517135620117	Acc: 0.18920021762785635
Epoch: 3	Loss: 2.4243907928466797	Acc: 0.18838411316648532
Epoch: 4	Loss: 2.2250819206237793	Acc: 0.1882480957562568
Epoch: 5	Loss: 2.067070484161377	Acc: 0.20389009793253537
Epoch: 6	Loss: 1.9329267740249634	Acc: 0.16648531011969533
Epoch: 7	Loss: 1.7174493074417114	Acc: 0.16376496191512513
Epoch: 8	Loss: 1.9999536275863647	Acc: 0.12023939064200218
Epoch: 9	Loss: 1.7145600318908691	Acc: 0.25068008705114253
Epoch: 10	Loss: 1.7372475862503052	Acc: 0.3041349292709467
Epoch: 11	Loss: 1.6658921241760254	Acc: 0.23993471164309033
Epoch: 12	Loss: 1.6845120191574097	Acc: 0.22320457018498369
Epoch: 13	Loss: 1.7191433906555176	Acc: 0.20837867247007616
Epoch: 14	Loss: 1.707417607307434	Acc: 0.2146354733405876
Epoch: 15	Loss: 1.6619631052017212	Acc: 0.2310935799782372
Epoch: 16	Loss: 1.6326324939727783	Acc: 0.3216811751904244
Epoch: 17	Loss: 1.6402760744094849	Acc: 0.342899891186

KeyboardInterrupt: 

In [ ]:
y_pred_test = model(testX)
acc_test = float((testy_c.long().eq(torch.max(y_pred_test, 1)[1].long())).sum())/(float(testy_c.shape[0]))
print("TestAcc: {}".format(acc_test))

In [ ]:
testX.shape

In [ ]:
testX.view((128,9,2947))

In [ ]:
testX.sha

In [ ]:
sample = torch.tensor([np.array(testX[0].cpu())]).cuda()

In [ ]:
testy[0]

In [ ]:
y_true[0]

In [ ]:
y_true

In [ ]:
y_pred

In [ ]:
y_true = testy.max(dim=1)[1][0].float()
y_pred = model.forward(sample).max(dim=1)[1].float()
criterion = nn.NLLLoss()
loss = criterion(torch.log(y_pred), y_true)
print(loss.item())

In [ ]:
torch.log(y_pred)

In [ ]:
trainy_c[0]

In [ ]:
trainy_c.unique()

In [ ]:
T = []
for i in range(trainX.shape[0]):
    T.append(trainX[i].T)
T = np.array(T)

In [ ]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred

In [ ]:
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Construct our model by instantiating the class defined above
model = TwoLayerNet(D_in, H, D_out)

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for t in range(500):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()